# Jupyter Notebook approach for SCORE interaction


### All the necessary packages to be imported

In [51]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from repeater import retry

### Uncomment the line which you want to use.

In [52]:
# Mainnet
# icon_service = IconService(HTTPProvider("https://ctz.solidwallet.io", 3))
# Euljiro
# icon_service = IconService(HTTPProvider("https://test-ctz.solidwallet.io",3))
# Yeouido
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
# Pagoda
# icon_service = IconService(HTTPProvider("https://zicon.net.solidwallet.io", 3))
# Custom
# icon_service = IconService(HTTPProvider("http://54.186.60.134:9000/", 3))

### NIDs

In [53]:
# Mainnet
# NID = 1
# Euljiro Testnet
# NID = 2
# Yeouido Testnet // Tbears
NID = 3
# Pagoda Testnet
# NID = 80

### List necessary SCORE addresses in this section

In [87]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

# Yeouido Testnet
SCORE_ADDRESS = 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6'

# Mainnet
# SCORE_ADDRESS = "cx346a2c30ea2712557deb4d7392688750350d1dea"

### Import the necessary wallets

If you want to use your pre-exisiting wallets, execute the cell below:-

In [55]:
deployer_wallet = KeyWallet.create()
print("deployer address: ", deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print("caller address: ", caller_wallet.get_address())

deployer address:  hx0eb8ed48c1f9e95a667208c1c790a91ba62a1255
caller address:  hx8f15a587ba5fce24af0415aa87deb58fa0b7e858


In [79]:
caller_wallet_address = "hx8f15a587ba5fce24af0415aa87deb58fa0b7e858"
deployer_wallet_address = "hx0eb8ed48c1f9e95a667208c1c790a91ba62a1255"

### Load ICX
Go to [ibriz faucet](https://icon-faucet.ibriz.ai/) to load ICX into your wallets.

### Deploying contract

In [65]:
#deploying of tokens
DEPLOY_PARAMS =  {
            # "_initialSupply": 1000,
            # "_decimals": 10
        }
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('nft.zip'))\
    .params(DEPLOY_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xea0c6f24c5b687c5707baca129307e229e7a17f04f7c9aaf76adf8f4ed0dd7b4',
 'blockHeight': 9065393,
 'blockHash': '0x35c007998e873c3cae3e7427c7b3dfd15fdfa0de736edd78a3cd6fec23dfd944',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
 'stepUsed': 1076261600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1076261600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [66]:
#getting the information of tokens
external_methods = ["name", "symbol"]
for method in external_methods:
    call = CallBuilder().from_(deployer_wallet.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method)\
                    .build()
    print(icon_service.call(call))

NonFungibleToken
NNFT


In [76]:
#getting nft token owned by owner and caller
external_methods = [deployer_wallet.get_address(), caller_wallet.get_address()]
for address in external_methods:
    call = CallBuilder().from_(deployer_wallet.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'_owner': address})\
                    .build()
    print(icon_service.call(call))

0x0
0x0


In [90]:
#minting the new nnft tokens
params={
    "_to": caller_wallet.get_address(),
    "_tokenId": 4,
}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet_address)\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("mint")\
    .params(params)\
    .build()


estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(call_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

tx_result = get_tx_result(tx_hash)
tx_result


{'txHash': '0x32adb18a15e80bce471894a4333a803b208785512e5f98190f132f2a5f0125c7',
 'blockHeight': 9065652,
 'blockHash': '0x4908634a1cfbf2f77a8e70d9161f6872e04d263b22875a7a75deda2e08233faf',
 'txIndex': 1,
 'to': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
 'stepUsed': 157880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 157880,
 'eventLogs': [{'scoreAddress': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
   'indexed': ['Transfer(Address,Address,int)',
    'hx0000000000000000000000000000000000000000',
    'hx8f15a587ba5fce24af0415aa87deb58fa0b7e858',
    '0x3'],
   'data': []}],
 'logsBloom': b'\x04\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x0

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x4c703d36f63e3077cfab2822dfed936b4e4009b1cbbc7332f0ed866521e92993',
 'blockHeight': 9065654,
 'blockHash': '0x5c94d7a2d1627584d4990e5380c81b0f7aac29d244600d927d1e8a7f11e505c1',
 'txIndex': 1,
 'to': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
 'stepUsed': 157880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 157880,
 'eventLogs': [{'scoreAddress': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
   'indexed': ['Transfer(Address,Address,int)',
    'hx0000000000000000000000000000000000000000',
    'hx8f15a587ba5fce24af0415aa87deb58fa0b7e858',
    '0x4'],
   'data': []}],
 'logsBloom': b'\x04\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x0

In [91]:
#getting nft token owned by owner and caller
external_methods = [deployer_wallet.get_address(), caller_wallet.get_address()]
for address in external_methods:
    call = CallBuilder().from_(deployer_wallet.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'_owner': address})\
                    .build()
    print(icon_service.call(call))

0x0
0x3


In [94]:
#getting token approved address
params={
    "_tokenId": 3,
}

call_transaction = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .method("getApproved")\
    .params(params)\
    .build()


print(icon_service.call(call_transaction))

hx0000000000000000000000000000000000000000


In [100]:
#approving token
params={
    "_to": deployer_wallet_address,
    "_tokenId": 2,
}

call_transaction = CallBuilder()\
    .from_(caller_wallet_address)\
    .to(SCORE_ADDRESS)\
    .method("approve")\
    .params(params)\
    .build()


print(icon_service.call(call_transaction))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32009, 'message': 'No permission to write'}


JSONRPCException: {'code': -32009, 'message': 'No permission to write'} (Json rpc error)

In [101]:
#transfer of token from caller to deployer
params={
    "_to": deployer_wallet_address,
    "_tokenId": 2,
}

call_transaction = CallTransactionBuilder()\
    .from_(caller_wallet_address)\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()


estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(call_transaction, caller_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

tx_result = get_tx_result(tx_hash)
tx_result

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xf462ce21ae9b92d8a0ccf8cff9fc0f1644be221856c5ae4b9f05ae9d4e07a56d',
 'blockHeight': 9065822,
 'blockHash': '0x355d1d020c062c57291ed084a0b6d942edd8657b9f6e6a75246ccabeaa8bbced',
 'txIndex': 1,
 'to': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
 'stepUsed': 155800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 155800,
 'eventLogs': [{'scoreAddress': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
   'indexed': ['Transfer(Address,Address,int)',
    'hx8f15a587ba5fce24af0415aa87deb58fa0b7e858',
    'hx0eb8ed48c1f9e95a667208c1c790a91ba62a1255',
    '0x2'],
   'data': []}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x0

In [102]:
#burning nft token

params = {
    '_tokenId': 3,
}

call_transaction = CallTransactionBuilder()\
    .from_(caller_wallet_address)\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(2321)\
    .method("burn")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(call_transaction, caller_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

tx_result = get_tx_result(tx_hash)
tx_result

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x949a4522b5b97af622f2336052e96763bc19eda110e2a28cefeae3bce7f6ed7f',
 'blockHeight': 9065830,
 'blockHash': '0x744256e75f931289222daa53507572c246fdaf9e6e26c836bb814df6360937ff',
 'txIndex': 1,
 'to': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
 'stepUsed': 142880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 142880,
 'eventLogs': [{'scoreAddress': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
   'indexed': ['Transfer(Address,Address,int)',
    'hx8f15a587ba5fce24af0415aa87deb58fa0b7e858',
    'hx0000000000000000000000000000000000000000',
    '0x3'],
   'data': []}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x

In [107]:
#gettig owner of token id
params = {
    '_tokenId': 4,
}

call_transaction = CallBuilder()\
    .from_(caller_wallet_address)\
    .to(SCORE_ADDRESS)\
    .method("ownerOf")\
    .params(params)\
    .build()
print(icon_service.call(call_transaction))

hx0eb8ed48c1f9e95a667208c1c790a91ba62a1255


In [104]:
#getting approved address of nft
params = {
    '_tokenId': 4,
}

call_transaction = CallBuilder()\
    .from_(deployer_wallet_address)\
    .to(SCORE_ADDRESS)\
    .method("getApproved")\
    .params(params)\
    .build()

print(icon_service.call(call_transaction))

hx0000000000000000000000000000000000000000


In [105]:
#gettig owner of token id
params = {
    '_from': caller_wallet_address,
    '_to': deployer_wallet_address,
    '_tokenId': 4,
}

call_transaction = CallTransactionBuilder()\
    .from_(caller_wallet_address)\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(2321)\
    .method("transferFrom")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(call_transaction, caller_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

tx_result = get_tx_result(tx_hash)
tx_result


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x5097a90ae983f3147ba6de5e66ced7791fc3a1bc67914ae195498381a26d38d6',
 'blockHeight': 9065932,
 'blockHash': '0xf5714cbc47e8e37411e85514b75d29af54897c1f56145cdd0f8620d0204cdeba',
 'txIndex': 1,
 'to': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
 'stepUsed': 166960,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 166960,
 'eventLogs': [{'scoreAddress': 'cx5408b39ff0b64325a118b88df5edbe4dd7a623b6',
   'indexed': ['Transfer(Address,Address,int)',
    'hx8f15a587ba5fce24af0415aa87deb58fa0b7e858',
    'hx0eb8ed48c1f9e95a667208c1c790a91ba62a1255',
    '0x4'],
   'data': []}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x04\x